### Step #1: We are going to fetch a list of countries from wikipedia and their respective links

In [447]:
# Import Beautifulsoup and other libraries. In this exercise Beautifulsoup will suffice
from datetime import datetime, timedelta
import requests
from bs4 import BeautifulSoup 
import pandas as pd
import re
import winsound
import sys
from selenium import webdriver
# from msedge.selenium_tools import Edge, EdgeOptions
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException, ElementClickInterceptedException, StaleElementReferenceException

options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
options.add_argument('--headless')
PATH = r'C:\Users\Emmanuel.Kimbe\Documents\chromedriver.exe'
driver = webdriver.Chrome(PATH, options=options)


heads_of_state_df = pd.DataFrame(columns =['HeadID','Name','Link','Country'])
countries_df = pd.DataFrame(columns =['Country_ID','Country','Wiki_Link'])
base_url = r'https://en.wikipedia.org'

In [448]:

#THere are several pages in Wikipedia that show list of countries, we will use the one with links to country pages
#https://en.wikipedia.org/wiki/List_of_sovereign_states - UN Members
#We will delete columns we do not need and remain with two columns: country name and its wikipedia link
URL = r'https://en.wikipedia.org/wiki/List_of_sovereign_states'
print('getting url')
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')
#table =soup.find(class_='wikitable sortable plainrowheaders jquery-tablesorter')
table =soup.find(class_='sortable wikitable')
rows = table.find_all('tr')

counter=0
for row in rows:
    
    country_info = row.find('a') #to get the first cell only
    if country_info != None:
        cells = row.find_all('td')
        
        if len(cells) == 0:
            
            continue
        
        membership = cells[1].text
        if membership.strip() == 'A UN member state':  
           
            country_link =base_url + country_info['href']
            country_name = country_info.text
            countries_df.loc[counter] = [counter, country_name, country_link]
#             print(counter,',',country_name,',',country_link)
            counter+=1
#countries_df = countries_df.set_index('Country_ID')
countries_df
        

getting url


,Country_ID,Country,Wiki_Link
0,0,Abkhazia,https://en.wikipedia.org#Abkhazia
1,1,Afghanistan,https://en.wikipedia.org/wiki/Afghanistan
2,2,Albania,https://en.wikipedia.org/wiki/Albania
3,3,Algeria,https://en.wikipedia.org/wiki/Algeria
4,4,Andorra,https://en.wikipedia.org/wiki/Andorra
...,...,...,...
217,217,Venezuela,https://en.wikipedia.org/wiki/Venezuela
218,218,Vietnam,https://en.wikipedia.org/wiki/Vietnam
219,219,Yemen,https://en.wikipedia.org/wiki/Yemen
220,220,Zambia,https://en.wikipedia.org/wiki/Zambia


### The next step is to open a page of each of the countries and read the name and link of the president

#### To do this we will loop through the dataframe and open get the country link and open it with BS then get the right tag. To reduce repition of this task we will create a function that takes a country link and returns a name and a link for its 3 most recent presidents/heads_of_state

In [449]:
# links = countries_df['Wiki_Link']
# for link in links[90:100]:
#     presidents = get_info(link)
#     print(presidents)

In [478]:



URL1 = r'https://en.wikipedia.org/wiki/List_of_current_heads_of_state_and_government'
print('Fetching heads of state')
page = requests.get(URL1)
soup = BeautifulSoup(page.content,'html.parser')
table = soup.find(class_="wikitable plainrowheaders")

rows = table.find_all('tr')
row_index = 0
for row in rows:
    
    if row_index > 0: # to skip the headers
        try:
            th = row.find('th')
            country = th.text.strip()
        except:
            #print('----error occured---')
            country = 'Error'
            
           
        try:
            td = row.find('td')
            links = td.find_all('a')
            link_count = 0
            for link in links:
                if link_count == 1:
                    head_link = base_url + link['href']
                    head_of_state = link.text.strip()
                link_count += 1
            head_info = td.text.strip().replace(u'\xa0', u' ').split('–')  
            head_title = head_info[0].strip()  
#             head_of_state = head_info[1].strip()
            
        except:
            #print('----error occured---')
            head_of_state = 'Error'
            head_title = 'Error'
            head_link = 'Error'
            if country == 'Switzerland':                
                head_of_state = 'Simonetta Sommaruga'
                head_title = 'President'
                head_link = 'https://en.wikipedia.org/wiki/Simonetta_Sommaruga'
                
        heads_of_state_df.loc[row_index] = [row_index, head_of_state, head_link, country]
        print('Country: {0}\nHead of State: {1}\nTitle: {2} \nWiki Link: {3}'.format(country,head_of_state,head_title, head_link))
        print('------------------------------------------------------------\n')
        
    row_index += 1
                 
#heads_of_state_df = heads_of_state_df.set_index('HeadID')
heads_of_state_df

Fetching heads of state
Country: Afghanistan
Head of State: Ashraf Ghani
Title: President 
Wiki Link: https://en.wikipedia.org/wiki/Ashraf_Ghani
------------------------------------------------------------

Country: Albania
Head of State: Ilir Meta
Title: President 
Wiki Link: https://en.wikipedia.org/wiki/Ilir_Meta
------------------------------------------------------------

Country: Algeria
Head of State: Abdelmadjid Tebboune
Title: President 
Wiki Link: https://en.wikipedia.org/wiki/Abdelmadjid_Tebboune
------------------------------------------------------------

Country: Andorra
Head of State: Joan Enric Vives i Sicília
Title: Episcopal Co-Prince 
Wiki Link: https://en.wikipedia.org/wiki/Joan_Enric_Vives_i_Sic%C3%ADlia
------------------------------------------------------------

Country: Angola
Head of State: João Lourenço
Title: President 
Wiki Link: https://en.wikipedia.org/wiki/Jo%C3%A3o_Louren%C3%A7o
------------------------------------------------------------

Country: Anti

Country: Saudi Arabia
Head of State: Salman
Title: King 
Wiki Link: https://en.wikipedia.org/wiki/Salman_of_Saudi_Arabia
------------------------------------------------------------

Country: Senegal
Head of State: Macky Sall
Title: President 
Wiki Link: https://en.wikipedia.org/wiki/Macky_Sall
------------------------------------------------------------

Country: Serbia
Head of State: Aleksandar Vučić
Title: President 
Wiki Link: https://en.wikipedia.org/wiki/Aleksandar_Vu%C4%8Di%C4%87
------------------------------------------------------------

Country: Seychelles
Head of State: Danny Faure
Title: President 
Wiki Link: https://en.wikipedia.org/wiki/Danny_Faure
------------------------------------------------------------

Country: Sierra Leone
Head of State: Julius Maada Bio
Title: President 
Wiki Link: https://en.wikipedia.org/wiki/Julius_Maada_Bio
------------------------------------------------------------

Country: Singapore
Head of State: Halimah Yacob
Title: President 
Wiki Lin

,HeadID,Name,Link,Country
1,1,Ashraf Ghani,https://en.wikipedia.org/wiki/Ashraf_Ghani,Afghanistan
2,2,Ilir Meta,https://en.wikipedia.org/wiki/Ilir_Meta,Albania
3,3,Abdelmadjid Tebboune,https://en.wikipedia.org/wiki/Abdelmadjid_Tebb...,Algeria
4,4,Joan Enric Vives i Sicília,https://en.wikipedia.org/wiki/Joan_Enric_Vives...,Andorra
5,5,João Lourenço,https://en.wikipedia.org/wiki/Jo%C3%A3o_Louren...,Angola
...,...,...,...,...
162,162,Mirko Dolcini,https://en.wikipedia.org/wiki/Mirko_Dolcini,Error
183,183,Error,Error,Federal Council[κ]
184,184,President,https://en.wikipedia.org/wiki/President_of_the...,Error
185,185,Ueli Maurer,https://en.wikipedia.org/wiki/Ueli_Maurer,Error


In [487]:
#The challenge with looking into heads of states is that some governments the head of state is a Monarchy or another ceremonial leader
#This will therefore leave out the actual leaders who make decisions, therefore we have to fetch the heads of governmetns as welll, but will will add them in the same list of heads_of_state
#For this task that should not be a problem






URL1 = r'https://en.wikipedia.org/wiki/List_of_current_heads_of_state_and_government'
print('Fetching heads of government....')
page = requests.get(URL1)
soup = BeautifulSoup(page.content,'html.parser')
table = soup.find(class_="wikitable plainrowheaders")

rows = table.find_all('tr')
print('There are {} rows'.format(len(rows)))
row_index = 0
counter = 0
for row in rows:
    
    if counter > 0: # to skip the headers
        try:
            th = row.find('th')
            country = th.text.strip()
        except:
            #print('----error occured---')
            country = 'Error'
            
           
        try:
            tds = row.find_all('td')
            print('There are {} "<td>\'s"'.format(len(tds)))
            for td in tds:
                
                links = td.find_all('a')
                link_count = 0
                for link in links:
                    if link_count == 1:
                        head_link = base_url + link['href']
                        head_of_state = link.text.strip()
                    link_count += 1
                head_info = td.text.strip().replace(u'\xa0', u' ').split('–')  
                head_title = head_info[0].strip()  
                heads_of_state_df.loc[row_index] = [row_index, head_of_state, head_link, country]
                row_index += 1
#             head_of_state = head_info[1].strip()
            
        except:
            print('----error occured---')
            head_of_state = 'Error'
            head_title = 'Error'
            head_link = 'Error'
            if country == 'Switzerland':                
                head_of_state = 'Simonetta Sommaruga'
                head_title = 'President'
                head_link = 'https://en.wikipedia.org/wiki/Simonetta_Sommaruga'
                heads_of_state_df.loc[row_index] = [row_index, head_of_state, head_link, country]
                row_index += 1
        
        print('Country: {0}\nHead of State: {1}\nTitle: {2} \nWiki Link: {3}'.format(country,head_of_state,head_title, head_link))
        print('------------------------------------------------------------\n')
        
    
    counter += 1                 
#heads_of_state_df = heads_of_state_df.set_index('HeadID')
heads_of_state_df

Fetching heads of government....
There are 212 rows
There are 1 "<td>'s"
Country: Afghanistan
Head of State: Ashraf Ghani
Title: President 
Wiki Link: https://en.wikipedia.org/wiki/Ashraf_Ghani
------------------------------------------------------------

There are 2 "<td>'s"
Country: Albania
Head of State: Edi Rama
Title: Prime Minister 
Wiki Link: https://en.wikipedia.org/wiki/Edi_Rama
------------------------------------------------------------

There are 2 "<td>'s"
Country: Algeria
Head of State: Abdelaziz Djerad
Title: Prime Minister 
Wiki Link: https://en.wikipedia.org/wiki/Abdelaziz_Djerad
------------------------------------------------------------

There are 2 "<td>'s"
Country: Andorra
Head of State: Xavier Espot Zamora
Title: Head of Government 
Wiki Link: https://en.wikipedia.org/wiki/Xavier_Espot_Zamora
------------------------------------------------------------

There are 1 "<td>'s"
Country: Angola
Head of State: João Lourenço
Title: President 
Wiki Link: https://en.wikip


There are 1 "<td>'s"
Country: Indonesia
Head of State: Joko Widodo
Title: President 
Wiki Link: https://en.wikipedia.org/wiki/Joko_Widodo
------------------------------------------------------------

There are 2 "<td>'s"
Country: Iran
Head of State: Hassan Rouhani
Title: President 
Wiki Link: https://en.wikipedia.org/wiki/Hassan_Rouhani
------------------------------------------------------------

There are 2 "<td>'s"
Country: Iraq
Head of State: Mustafa Al-Kadhimi
Title: Prime Minister 
Wiki Link: https://en.wikipedia.org/wiki/Mustafa_Al-Kadhimi
------------------------------------------------------------

There are 2 "<td>'s"
Country: Ireland
Head of State: Micheál Martin
Title: Taoiseach 
Wiki Link: https://en.wikipedia.org/wiki/Miche%C3%A1l_Martin
------------------------------------------------------------

There are 2 "<td>'s"
Country: Israel
Head of State: Benjamin Netanyahu
Title: Prime Minister 
Wiki Link: https://en.wikipedia.org/wiki/Benjamin_Netanyahu
---------------------

Country: Error
Head of State: Kim Tok-hun
Title: Premier of the Cabinet 
Wiki Link: https://en.wikipedia.org/wiki/Kim_Tok-hun
------------------------------------------------------------

There are 2 "<td>'s"
Country: North Macedonia
Head of State: Zoran Zaev
Title: President of the Government 
Wiki Link: https://en.wikipedia.org/wiki/Zoran_Zaev
------------------------------------------------------------

There are 2 "<td>'s"
Country: Norway
Head of State: Erna Solberg
Title: Prime Minister 
Wiki Link: https://en.wikipedia.org/wiki/Erna_Solberg
------------------------------------------------------------

There are 1 "<td>'s"
Country: Oman
Head of State: Prime Minister
Title: Sultan and Prime Minister 
Wiki Link: https://en.wikipedia.org/wiki/Prime_Minister_of_Oman
------------------------------------------------------------

There are 2 "<td>'s"
Country: Pakistan
Head of State: Imran Khan
Title: Prime Minister 
Wiki Link: https://en.wikipedia.org/wiki/Imran_Khan
---------------------

Country: Syria
Head of State: Hussein Arnous
Title: Prime Minister 
Wiki Link: https://en.wikipedia.org/wiki/Hussein_Arnous
------------------------------------------------------------

There are 2 "<td>'s"
Country: Tajikistan
Head of State: Kokhir Rasulzoda
Title: Prime Minister 
Wiki Link: https://en.wikipedia.org/wiki/Kokhir_Rasulzoda
------------------------------------------------------------

There are 2 "<td>'s"
Country: Tanzania
Head of State: Kassim Majaliwa
Title: Prime Minister 
Wiki Link: https://en.wikipedia.org/wiki/Kassim_Majaliwa
------------------------------------------------------------

There are 2 "<td>'s"
Country: Thailand
Head of State: Prayut Chan-o-cha
Title: Prime Minister 
Wiki Link: https://en.wikipedia.org/wiki/Prayut_Chan-o-cha
------------------------------------------------------------

There are 2 "<td>'s"
Country: Togo
Head of State: Victoire Tomegah Dogbé
Title: Prime Minister 
Wiki Link: https://en.wikipedia.org/wiki/Victoire_Tomegah_Dogb%C3%A9
-----

,HeadID,Name,Link,Country
1,1,Ilir Meta,https://en.wikipedia.org/wiki/Ilir_Meta,Albania
2,2,Edi Rama,https://en.wikipedia.org/wiki/Edi_Rama,Albania
3,3,Abdelmadjid Tebboune,https://en.wikipedia.org/wiki/Abdelmadjid_Tebb...,Algeria
4,4,Abdelaziz Djerad,https://en.wikipedia.org/wiki/Abdelaziz_Djerad,Algeria
5,5,Joan Enric Vives i Sicília,https://en.wikipedia.org/wiki/Joan_Enric_Vives...,Andorra
...,...,...,...,...
344,344,Nguyễn Xuân Phúc,https://en.wikipedia.org/wiki/Nguy%E1%BB%85n_X...,Error
345,345,Abdrabbuh Mansur Hadi,https://en.wikipedia.org/wiki/Abdrabbuh_Mansur...,Yemen
346,346,Maeen Abdulmalik Saeed,https://en.wikipedia.org/wiki/Maeen_Abdulmalik...,Yemen
347,347,Edgar Lungu,https://en.wikipedia.org/wiki/Edgar_Lungu,Zambia


In [488]:
len(heads_of_state_df.index)

349

#### Let's inspect the data and see how many rows contain "'Error's"


In [489]:
#First let's look at those with error in the 'Name, Column
name_error_df = heads_of_state_df[heads_of_state_df['Name']=='Error']
name_error_df
#After running the first test you see Switzerland and 'Federal...'

,HeadID,Name,Link,Country


In [490]:

# Secondly let's look at those with error in the 'Link' Column
link_error_df = heads_of_state_df[heads_of_state_df['Link']=='Error']
link_error_df
#Again, it's the same as above



,HeadID,Name,Link,Country


In [491]:
# #Next let's look at those with error in the 'Country' column
country_error_df = heads_of_state_df[heads_of_state_df['Country']=='Error']
country_error_df

#Then inspect the row with Switzerland: Data structure of 'Switzerland's row is different from others, it has two <th>'s
#The data for this country is relatively randomly arranged, it will be fine for now to treat it as an exception and create a different way of handling it without compromising the others


,HeadID,Name,Link,Country
39,39,Chairman,https://en.wikipedia.org/wiki/Chairman_of_the_...,Error
63,63,Xi Jinping,https://en.wikipedia.org/wiki/Xi_Jinping,Error
64,64,Li Keqiang,https://en.wikipedia.org/wiki/Li_Keqiang,Error
75,75,Miguel Díaz-Canel,https://en.wikipedia.org/wiki/Miguel_D%C3%ADaz...,Error
76,76,Manuel Marrero Cruz,https://en.wikipedia.org/wiki/Manuel_Marrero_Cruz,Error
96,96,Isaias Afwerki,https://en.wikipedia.org/wiki/Isaias_Afwerki,Error
157,157,Qasym-Zhomart Toqaev,https://en.wikipedia.org/wiki/Kassym-Jomart_To...,Error
158,158,Askar Mamin,https://en.wikipedia.org/wiki/Askar_Mamin,Error
166,166,Bounnhang Vorachith,https://en.wikipedia.org/wiki/Bounnhang_Vorachith,Error
167,167,Thongloun Sisoulith,https://en.wikipedia.org/wiki/Thongloun_Sisoulith,Error


#### The next part is to open the links and get the predicessors table

#### But before proceeding we have to clean all rows where there is error in the country column. This can easily be done with pandas

In [492]:
country_error_bool =heads_of_state_df['Country']!='Error' 
link_error_bool =  heads_of_state_df['Link']!='Error'
heads_of_state_df = heads_of_state_df[link_error_bool]
heads_of_state_df = heads_of_state_df[country_error_bool]
pred_heads_df1 = heads_of_state.copy(deep = False)
counter = len(heads_of_state_df)

In [493]:
def valid_url(url):
    regex = re.compile(
        r'^(?:http|ftp)s?://' # http:// or https://
        r'(?:(?:[A-Z0-9](?:[A-Z0-9-]{0,61}[A-Z0-9])?\.)+(?:[A-Z]{2,6}\.?|[A-Z0-9-]{2,}\.?)|' #domain...
        r'localhost|' #localhost...
        r'\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3})' # ...or ip
        r'(?::\d+)?' # optional port
        r'(?:/?|[/?]\S+)$', re.IGNORECASE)
    return re.match(regex,url)


In [494]:
#create a function for pulling a predicessor, given the link of the incumbent
def pred_head(link):
    page = requests.get(link)
    soup = BeautifulSoup(page.content, 'html.parser')
  
    trs = soup.find_all('tr')
    count_th = 0    
    for tr in trs:
        try:
            th = tr.find('th')
            if th.text == 'Preceded by' or th.text == 'Predecessor':
                if count_th == 0:
                    td = tr.find('td')
                    if td.text != None:
                        if td.text == 'Position established':
                            pred_link = td.find('a')
                            link_pred = base_url + pred_link['href'].strip()
                            name_pred = pred_link.text.strip()
                        else:
                            
                            link_pred = 'None'#base_url + pred_link['href'].strip()
                            name_pred = 'Position established' #pred_link.text.strip()
                            
                        pred_link = td.find('a')
                        link_pred = base_url + pred_link['href'].strip()
                        name_pred = pred_link.text.strip()
                        return link_pred, name_pred
                count_th+=1
        except:
            pass
#             e0 = sys.exc_info()[0]
#             e1 = sys.exc_info()[1]
#             print(e0,'\n',e1)
#             return ('Error','Error')

In [495]:
#iterate through all links and open one-by-one

row_index = 0

for index,row in heads_of_state_df.iterrows():
    link = row['Link']
    if valid_url(link) == None:
        continue
    country = row['Country']
    info = pred_head(link)
    try:
        pred_link = info[0]
        pred_name = info[1]
        pred_heads_df1.loc[row_index] = [row_index, pred_name, pred_link, country]
    except:
        pass
    print(row['Name'],'Predicessor>>>',  info)
    row_index +=1
pred_heads_df1    

Ilir Meta Predicessor>>> ('https://en.wikipedia.org/wiki/Bujar_Nishani', 'Bujar Nishani')
Edi Rama Predicessor>>> ('https://en.wikipedia.org/wiki/Sali_Berisha', 'Sali Berisha')
Abdelmadjid Tebboune Predicessor>>> ('https://en.wikipedia.org/wiki/Abdelaziz_Bouteflika', 'Abdelaziz Bouteflika')
Abdelaziz Djerad Predicessor>>> ('https://en.wikipedia.org/wiki/Sabri_Boukadoum', 'Sabri Boukadoum')
Joan Enric Vives i Sicília Predicessor>>> ('https://en.wikipedia.org/wiki/Joan_Mart%C3%AD_i_Alanis', 'Joan Martí i Alanis')
Xavier Espot Zamora Predicessor>>> ('https://en.wikipedia.org/wiki/Antoni_Mart%C3%AD', 'Antoni Martí')
João Lourenço Predicessor>>> ('https://en.wikipedia.org/wiki/Jos%C3%A9_Eduardo_dos_Santos', 'José Eduardo dos Santos')
Elizabeth II Predicessor>>> ('https://en.wikipedia.org/wiki/George_VI', 'George VI')
Gaston Browne Predicessor>>> ('https://en.wikipedia.org/wiki/Baldwin_Spencer', 'Baldwin Spencer')
Alberto Fernández Predicessor>>> ('https://en.wikipedia.org/wiki/Mauricio_Macr

Teodoro Obiang Nguema Mbasogo Predicessor>>> ('https://en.wikipedia.org/wiki/Francisco_Mac%C3%ADas_Nguema', 'Francisco Macías Nguema')
Francisco Pascual Obama Asue Predicessor>>> ('https://en.wikipedia.org/wiki/Vicente_Ehate_Tomi', 'Vicente Ehate Tomi')
Kersti Kaljulaid Predicessor>>> ('https://en.wikipedia.org/wiki/Toomas_Hendrik_Ilves', 'Toomas Hendrik Ilves')
Jüri Ratas Predicessor>>> ('https://en.wikipedia.org/wiki/Taavi_R%C3%B5ivas', 'Taavi Rõivas')
Mswati III Predicessor>>> ('https://en.wikipedia.org/wiki/Sobhuza_II', 'Sobhuza II')
Ambrose Mandvulo Dlamini Predicessor>>> ('https://en.wikipedia.org/wiki/Vincent_Mhlanga', 'Vincent Mhlanga')
Abiy Ahmed Predicessor>>> ('https://en.wikipedia.org/wiki/Hailemariam_Desalegn', 'Hailemariam Desalegn')
Jioji Konrote Predicessor>>> ('https://en.wikipedia.org/wiki/Epeli_Nailatikau', 'Epeli Nailatikau')
Frank Bainimarama Predicessor>>> ('https://en.wikipedia.org/wiki/Jona_Senilagakali', 'Jona Senilagakali')
Sauli Niinistö Predicessor>>> ('http

Bah Ndaw Predicessor>>> ('https://en.wikipedia.org/wiki/Assimi_Go%C3%AFta', 'Assimi Goïta')
Moctar Ouane Predicessor>>> ('https://en.wikipedia.org/wiki/Boubou_Ciss%C3%A9', 'Boubou Cissé')
George Vella Predicessor>>> ('https://en.wikipedia.org/wiki/Marie-Louise_Coleiro_Preca', 'Marie-Louise Coleiro Preca')
Robert Abela Predicessor>>> ('https://en.wikipedia.org/wiki/Joseph_Muscat', 'Joseph Muscat')
David Kabua Predicessor>>> ('https://en.wikipedia.org/wiki/Hilda_Heine', 'Hilda Heine')
Mohamed Ould Ghazouani Predicessor>>> ('https://en.wikipedia.org/wiki/Mohamed_Ould_Abdel_Aziz', 'Mohamed Ould Abdel Aziz')
Mohamed Ould Bilal Predicessor>>> ('https://en.wikipedia.org/wiki/Ismail_Ould_Bedde_Ould_Cheikh_Sidiya', 'Ismail Ould Bedde Ould Cheikh Sidiya')
Prithvirajsing Roopun Predicessor>>> ('https://en.wikipedia.org/wiki/Ameenah_Gurib', 'Ameenah Gurib')
Pravind Jugnauth Predicessor>>> ('https://en.wikipedia.org/wiki/Anerood_Jugnauth', 'Anerood Jugnauth')
Andrés Manuel López Obrador Predicessor

Paul Kagame Predicessor>>> ('https://en.wikipedia.org/wiki/Pasteur_Bizimungu', 'Pasteur Bizimungu')
Édouard Ngirente Predicessor>>> ('https://en.wikipedia.org/wiki/Anastase_Murekezi', 'Anastase Murekezi')
Elizabeth II Predicessor>>> ('https://en.wikipedia.org/wiki/George_VI', 'George VI')
Timothy Harris Predicessor>>> ('https://en.wikipedia.org/wiki/Denzil_Douglas', 'Denzil Douglas')
Elizabeth II Predicessor>>> ('https://en.wikipedia.org/wiki/George_VI', 'George VI')
Allen Chastanet Predicessor>>> ('https://en.wikipedia.org/wiki/Kenny_Anthony', 'Kenny Anthony')
Elizabeth II Predicessor>>> ('https://en.wikipedia.org/wiki/George_VI', 'George VI')
Ralph Gonsalves Predicessor>>> ('https://en.wikipedia.org/wiki/Arnhim_Eustace', 'Arnhim Eustace')
Va'aletoa Sualauvi II Predicessor>>> ('https://en.wikipedia.org/wiki/Tui_Atua_Tupua_Tamasese_Efi', 'Tui Atua Tupua Tamasese Efi')
Tuilaepa Aiono Sailele Malielegaoi Predicessor>>> ('https://en.wikipedia.org/wiki/Tofilau_Eti_Alesana', 'Tofilau Eti Al

Maeen Abdulmalik Saeed Predicessor>>> ('https://en.wikipedia.org/wiki/Ahmed_Obeid_bin_Daghr', 'Ahmed Obeid bin Daghr')
Edgar Lungu Predicessor>>> ('https://en.wikipedia.org/wiki/Guy_Scott', 'Guy Scott')
Emmerson Mnangagwa Predicessor>>> ('https://en.wikipedia.org/wiki/Robert_Mugabe', 'Robert Mugabe')


,HeadID,Name,Link,Country
0,0,Bujar Nishani,https://en.wikipedia.org/wiki/Bujar_Nishani,Albania
1,1,Sali Berisha,https://en.wikipedia.org/wiki/Sali_Berisha,Albania
2,2,Abdelaziz Bouteflika,https://en.wikipedia.org/wiki/Abdelaziz_Boutef...,Algeria
3,3,Sabri Boukadoum,https://en.wikipedia.org/wiki/Sabri_Boukadoum,Algeria
4,4,Joan Martí i Alanis,https://en.wikipedia.org/wiki/Joan_Mart%C3%AD_...,Andorra
...,...,...,...,...
324,324,Nông Đức Mạnh,https://en.wikipedia.org/wiki/N%C3%B4ng_%C4%90...,Vietnam
325,325,Ali Abdullah Saleh,https://en.wikipedia.org/wiki/Ali_Abdullah_Saleh,Yemen
326,326,Ahmed Obeid bin Daghr,https://en.wikipedia.org/wiki/Ahmed_Obeid_bin_...,Yemen
327,327,Guy Scott,https://en.wikipedia.org/wiki/Guy_Scott,Zambia


#### Then we repeat the procedure above by using the new dataframe consisting of the latest predecessors 

In [496]:
#iterate through all links and open one-by-one
country_error_bool =pred_heads_df1['Country']!='Error' 
link_error_bool =  pred_heads_df1['Link']!='Error'

pred_heads_df1 = pred_heads_df1[link_error_bool]
pred_heads_df1 = pred_heads_df1[country_error_bool]
pred_heads_df2 = pred_heads_df1.copy(deep = False)
counter = len(heads_of_state_df) + len(pred_heads_df1)

row_index = 0

for index,row in pred_heads_df1.iterrows():
    link = row['Link']
    if valid_url(link) == None:
        continue
    country = row['Country']
    info = pred_head(link)
    try:
        pred_link = info[0]
        pred_name = info[1]
        pred_heads_df2.loc[row_index] = [row_index, pred_name, pred_link, country]
    except:
        pass
    print(row['Name'],'Predicessor>>>', info)
    row_index +=1
pred_heads_df2   

Bamir Topi Predicessor>>> ('https://en.wikipedia.org/wiki/Bamir_Topi', 'Bamir Topi')
Fatos Nano Predicessor>>> ('https://en.wikipedia.org/wiki/Fatos_Nano', 'Fatos Nano')
Liamine Zéroual Predicessor>>> ('https://en.wikipedia.org/wiki/Liamine_Z%C3%A9roual', 'Liamine Zéroual')
Ramtane Lamamra Predicessor>>> ('https://en.wikipedia.org/wiki/Ramtane_Lamamra', 'Ramtane Lamamra')
Ramón Iglesias Navarri Predicessor>>> ('https://en.wikipedia.org/wiki/Ramon_Iglesias_i_Navarri', 'Ramón Iglesias Navarri')
Gilbert Saboya Sunyé Predicessor>>> ('https://en.wikipedia.org/wiki/Gilbert_Saboya_Suny%C3%A9', 'Gilbert Saboya Sunyé')
Lúcio Lara Predicessor>>> ('https://en.wikipedia.org/wiki/L%C3%BAcio_Lara', 'Lúcio Lara')
Edward VIII Predicessor>>> ('https://en.wikipedia.org/wiki/Edward_VIII', 'Edward VIII')
Lester Bird Predicessor>>> ('https://en.wikipedia.org/wiki/Lester_Bird', 'Lester Bird')
Cristina Fernández de Kirchner Predicessor>>> ('https://en.wikipedia.org/wiki/Cristina_Fern%C3%A1ndez_de_Kirchner', 

Taavi Rõivas Predicessor>>> ('https://en.wikipedia.org/wiki/Andrus_Ansip', 'Andrus Ansip')
Sobhuza II Predicessor>>> ('https://en.wikipedia.org/wiki/Ngwane_V_of_Swaziland', 'Ngwane V')
Vincent Mhlanga Predicessor>>> ('https://en.wikipedia.org/wiki/Barnabas_Sibusiso_Dlamini', 'Barnabas Sibusiso Dlamini')
Hailemariam Desalegn Predicessor>>> ('https://en.wikipedia.org/wiki/Meles_Zenawi', 'Meles Zenawi')
Epeli Nailatikau Predicessor>>> ('https://en.wikipedia.org/wiki/Jiko_Luveni', 'Jiko Luveni')
Jona Senilagakali Predicessor>>> ('https://en.wikipedia.org/wiki/Laisenia_Qarase', 'Laisenia Qarase')
Tarja Halonen Predicessor>>> ('https://en.wikipedia.org/wiki/Martti_Ahtisaari', 'Martti Ahtisaari')
Antti Rinne Predicessor>>> ('https://en.wikipedia.org/wiki/Tuula_Haatainen', 'Tuula Haatainen')
François Hollande Predicessor>>> ('https://en.wikipedia.org/wiki/Nicolas_Sarkozy', 'Nicolas Sarkozy')
Omar Bongo Predicessor>>> ('https://en.wikipedia.org/wiki/L%C3%A9on_M%27ba', "Léon M'ba")
Julien Nkoghe

Ismail Ould Bedde Ould Cheikh Sidiya Predicessor>>> ('https://en.wikipedia.org/wiki/Mohamed_Salem_Ould_B%C3%A9chir', 'Mohamed Salem Ould Béchir')
Ameenah Gurib Predicessor>>> ('https://en.wikipedia.org/wiki/Monique_Ohsan_Bellepeau', 'Monique Ohsan Bellepeau')
Anerood Jugnauth Predicessor>>> ('https://en.wikipedia.org/wiki/Navinchandra_Ramgoolam', 'Navinchandra Ramgoolam')
Enrique Peña Nieto Predicessor>>> ('https://en.wikipedia.org/wiki/Felipe_Calder%C3%B3n', 'Felipe Calderón')
Peter M. Christian Predicessor>>> ('https://en.wikipedia.org/wiki/Manny_Mori', 'Manny Mori')
Nicolae Timofti Predicessor>>> ('https://en.wikipedia.org/wiki/Marian_Lupu', 'Marian Lupu')
Maia Sandu Predicessor>>> ('https://en.wikipedia.org/wiki/Pavel_Filip', 'Pavel Filip')
Rainier III Predicessor>>> ('https://en.wikipedia.org/wiki/Louis_II,_Prince_of_Monaco', 'Louis II')
Serge Telle Predicessor>>> ('https://en.wikipedia.org/wiki/Gilles_Tonelli', 'Gilles Tonelli')
Tsakhiagiin Elbegdorj Predicessor>>> ('https://en.w

Tui Atua Tupua Tamasese Efi Predicessor>>> ('https://en.wikipedia.org/wiki/Malietoa_Tanumafili_II', 'Tanumafili II')
Tofilau Eti Alesana Predicessor>>> ('https://en.wikipedia.org/wiki/Va%27ai_Kolone', "Va'ai Kolone")
Alessandro Mancini Predicessor>>> ('https://en.wikipedia.org/wiki/Luca_Boschi', 'Luca Boschi')
Manuel Pinto da Costa Predicessor>>> ('https://en.wikipedia.org/wiki/Fradique_de_Menezes', 'Fradique de Menezes')
Patrice Trovoada Predicessor>>> ('https://en.wikipedia.org/wiki/Gabriel_Costa', 'Gabriel Costa')
Abdullah Predicessor>>> ('https://en.wikipedia.org/wiki/Fahd_of_Saudi_Arabia', 'Fahd')
Muhammad bin Nayef Predicessor>>> ('https://en.wikipedia.org/wiki/Muqrin_bin_Abdulaziz', 'Muqrin bin Abdulaziz')
Abdoulaye Wade Predicessor>>> ('https://en.wikipedia.org/wiki/Abdou_Diouf', 'Abdou Diouf')
Tomislav Nikolić Predicessor>>> ('https://en.wikipedia.org/wiki/Slavica_%C4%90uki%C4%87_Dejanovi%C4%87', 'Slavica Đukić Dejanović')
Aleksandar Vučić Predicessor>>> ('https://en.wikipedia

,HeadID,Name,Link,Country
0,0,Bamir Topi,https://en.wikipedia.org/wiki/Bamir_Topi,Albania
1,1,Fatos Nano,https://en.wikipedia.org/wiki/Fatos_Nano,Albania
2,2,Liamine Zéroual,https://en.wikipedia.org/wiki/Liamine_Z%C3%A9r...,Algeria
3,3,Ramtane Lamamra,https://en.wikipedia.org/wiki/Ramtane_Lamamra,Algeria
4,4,Ramón Iglesias Navarri,https://en.wikipedia.org/wiki/Ramon_Iglesias_i...,Andorra
...,...,...,...,...
38,38,Boyko Borisov,https://en.wikipedia.org/wiki/Boyko_Borisov,Bulgaria
153,153,Valdas Adamkus,https://en.wikipedia.org/wiki/Valdas_Adamkus,Lithuania
220,220,Yasser Arafat,https://en.wikipedia.org/wiki/Yasser_Arafat,Palestine
263,263,Janez Drnovšek,https://en.wikipedia.org/wiki/Janez_Drnov%C5%A1ek,Slovenia


In [497]:
# link_test = heads_of_state_df[heads_of_state_df['Link']=='Error']
# link_test

In [498]:
all_heads_df = pd.concat([heads_of_state_df, pred_heads_df1, pred_heads_df2])
all_heads_df = all_heads_df.drop_duplicates(subset=['Link'])
all_heads_df

,HeadID,Name,Link,Country
1,1,Ilir Meta,https://en.wikipedia.org/wiki/Ilir_Meta,Albania
2,2,Edi Rama,https://en.wikipedia.org/wiki/Edi_Rama,Albania
3,3,Abdelmadjid Tebboune,https://en.wikipedia.org/wiki/Abdelmadjid_Tebb...,Algeria
4,4,Abdelaziz Djerad,https://en.wikipedia.org/wiki/Abdelaziz_Djerad,Algeria
5,5,Joan Enric Vives i Sicília,https://en.wikipedia.org/wiki/Joan_Enric_Vives...,Andorra
...,...,...,...,...
21,21,Iajuddin Ahmed,https://en.wikipedia.org/wiki/Iajuddin_Ahmed,Bangladesh
153,153,Valdas Adamkus,https://en.wikipedia.org/wiki/Valdas_Adamkus,Lithuania
220,220,Yasser Arafat,https://en.wikipedia.org/wiki/Yasser_Arafat,Palestine
263,263,Janez Drnovšek,https://en.wikipedia.org/wiki/Janez_Drnov%C5%A1ek,Slovenia


### The next step is to open each link and extract the notes and references

In [500]:
links_fr={}
for index,row in all_heads_df.iterrows():
    link = row['Link']
    #print(link)
    if valid_url(link) != None:
        print(str(index),'Processing ....', link)
        page = requests.get(link)
        soup = BeautifulSoup(page.content, 'html.parser')
        references_list = soup.find_all(class_='references')
        print('Reference lists: {}'.format(len(references_list)))
        for references in references_list:
            
            try:
                refs = references.find_all('li')
                for ref in refs:
                    ref_links = ref.find_all('a')
                    for ref_link in ref_links:
                        href = ref_link['href'].strip()
                        if valid_url(href) != None:
                            url_base = href.split('/')[2]
                            if not url_base.startswith('www.'):
                                url_base = 'www.'+ url_base
                            if url_base in links_fr:
                                links_fr[url_base] += 1
                            else:
                                links_fr[url_base]=1
                            print(url_base)
            except:
                print('Error')
                pass
        #Then look for notes
        
#                     print(url_base)
#                     print(href)
freq_df = pd.DataFrame(links_fr.items(), columns = ['Source','Frequency'])                   
print(links_fr)
freq_df

1 Processing .... https://en.wikipedia.org/wiki/Ilir_Meta
Reference lists: 1
www.rtsh.al
www.parlament.al
www.telegrafi.com
www.thecelticstar.com
www.standard.al
www.revistawho.com
www.cls-sofia.org
www.web.archive.org
www.euractiv.com
www.rferl.org
www.pdc.ceu.hu
www.books.google.com
www.shtetiweb.org
www.taz.de
www.balkaneu.com
2 Processing .... https://en.wikipedia.org/wiki/Edi_Rama
Reference lists: 1
www.panorama.com.al
www.kryeministria.al
www.telegrafi.com
www.balkaninsight.com
www.gazetadita.al
www.lajmi.info
www.panorama.com.al
www.news.bbc.co.uk
www.creativetime.org
www.shekulli.com.al
www.sot.com.al
www.shqiptarja.com
www.news.bbc.co.uk
www.observers.france24.com
www.worldmayor.com
www.worldarchitecturenews.com
www.hdr.undp.org
www.illyriapress.com
www.arkivi2.peshkupauje.com
www.euobserver.com
www.setimes.com
www.euobserver.com
www.ps.al
www.punetebrendshme.gov.al
www.ata.gov.al
www.infocip.org
www.web.archive.org
www.euralius.eu
www.web.archive.org
www.kryeministria.al
www.

Reference lists: 2
www.rferl.org
www.tribune.com.pk
www.tcs.cam.ac.uk
www.weforum.org
www.gamefaqs.gamespot.com
www.showsotros.com
www.mobygames.com
www.panarmenian.net
www.armenpress.am
www.web.archive.org
www.eurasiahouse.com
www.aravot-en.am
www.mediamax.am
www.web.archive.org
www.EurasiaHouse.com
www.panorama.am
www.news.am
www.arka.am
www.web.archive.org
www.m.apa.az
www.armradio.am
www.rferl.org
www.rferl.org
www.armenianweekly.com
www.eadaily.com
www.agenda.ge
www.news.am
www.panorama.am
www.web.archive.org
www.voanews.com
www.old.armradio.am
www.arka.am
www.armenpress.am
www.armenpress.am
www.news.am
www.news.am
www.asbarez.com
www.president.am
www.president.am
www.asbarez.com
www.news.am
www.ru.armeniasputnik.am
www.ru.armeniasputnik.am
www.president.am
www.president.am
www.quirinale.it
www.armenpress.am
www.asbarez.com
12 Processing .... https://en.wikipedia.org/wiki/Nikol_Pashinyan
Reference lists: 1
www.edition.cnn.com
www.freedomhouse.org
www.eurasianet.org
www.web.faceboo

Reference lists: 1
www.heute.at
www.bbc.co.uk
www.courrierinternational.com
www.web.archive.org
www.gruene.at
www.orf.at
www.nytimes.com
www.independent.co.uk
www.theguardian.com
www.wahl16.bmi.gv.at
www.diepresse.com
www.gruene.at
www.theguardian.com
www.sonntagsblatt.at
www.web.archive.org
www.derstandard.at
www.wienerzeitung.at
www.aljazeera.com
www.theguardian.com
www.telegraph.co.uk
www.welt.de
www.washingtonpost.com
www.reuters.com
www.kurier.at
www.thelocal.at
www.krone.at
www.derstandard.at
www.whois.domaintools.com
www.youtube.com
www.nzz.ch
www.news.ORF.at
www.diepresse.com
www.archive.today
www.vanderbellen.at
www.archive.today
www.vanderbellen.at
www.diepresse.com
www.derstandard.at
www.orf.at
www.web.archive.org
www.wirtschaftsblatt.at
www.derstandard.at
www.derstandard.at
www.wahl16.bmi.gv.at
www.derstandard.at
www.sueddeutsche.de
www.parlament.gv.at
www.parlament.gv.at
www.quirinale.it
www.luxarazzi.com
16 Processing .... https://en.wikipedia.org/wiki/Sebastian_Kurz
Refe

Reference lists: 1
www.aljazeera.com
www.eurasianet.org
www.cabmin.gov.az
www.cabmin.gov.az
www.meclis.az
www.president.az
www.rferl.org
www.e-qanun.az
www.president.az
www.president.az
20 Processing .... https://en.wikipedia.org/wiki/Hubert_Minnis
Reference lists: 1
www.magneticmediatv.com
www.bahamaspress.com
www.tribune242.com
www.caribjournal.com
www.caribbean360.com
www.uwi.edu
www.wikielections.com
www.tribune242.com
www.tribune242.com
www.caribbean360.com
21 Processing .... https://en.wikipedia.org/wiki/Hamad_bin_Isa_Al_Khalifa
Reference lists: 1
www.acrli.org
www.web.archive.org
www.bahrainembassy.org
www.accessmylibrary.com
www.mofa.gov.bh
www.amnesty.org
www.files.bici.org.bh
www.interior.gov.bh
www.moj.gov.bh
www.mof.gov.bh
www.web.archive.org
www.moc.gov.bh
www.mofa.gov.bh
www.web.archive.org
www.bahrainedb.com
www.web.archive.org
www.scw.gov.bh
www.2009-2017.state.gov
www.bbc.co.uk
www.scoop.co.nz
www.independent.co.uk
www.theguardian.com
www.web.archive.org
www.foreignpol

Reference lists: 2
www.dhnet.be
www.rtbf.be
www.monarchie.be
www.monarchie.be
www.abh-ace.be
www.web.archive.org
www.web.archive.org
www.monarchie.be
www.bbc.co.uk
www.cnn.com
www.politico.eu
www.thepublicsradio.org
www.senate.be
www.kongehuset.dk
www.ejustice.just.fgov.be
www.ejustice.just.fgov.be
www.ejustice.just.fgov.be
www.brusselstimes.com
30 Processing .... https://en.wikipedia.org/wiki/Alexander_De_Croo
Reference lists: 1
www.wwwpr.belgium.be
www.hermandecroo.be
www.web.archive.org
www.alexanderdecroo.be
www.vrt.be
www.demorgen.be
www.standaard.be
www.thebulletin.be
www.theguardian.com
www.hln.be
www.radio1.be
www.msn.com
www.nieuwsblad.be
www.decroo.belgium.be
www.hln.be
www.vrt.be
www.aljazeera.com
www.reuters.com
www.shedecides.com
www.theguardian.com
www.vrt.be
www.diplomatie.belgium.be
www.reuters.com
www.knack.be
www.vrt.be
www.politico.eu
www.nieuwsblad.be
www.vrt.be
www.hln.be
www.gva.be
www.eib.org
www.esm.europa.eu
www.afdb.org
www.adb.org
www.ebrd.com
www.www3.weforu

...

Reference lists: 1
www.radiosvoboda.org
www.jpost.com
www.haaretz.com
www.timesofisrael.com
www.web.archive.org
www.kyivpost.com
www.web.archive.org
www.file.liga.net
www.bbc.com
www.vmr.gov.ua
www.coe.int
www.web.archive.org
www.heritageabroad.gov
www.pravda.com.ua
www.en.interfax.com.ua
www.rferl.org
www.jewishpress.com
www.timesofisrael.com
www.jta.org
www.m.dw.com
www.m.jpost.com
www.straitstimes.com
www.timesofindia.indiatimes.com
www.nytimes.com
www.en.interfax.com.ua
www.en.interfax.com.ua
www.en.interfax.com.ua
www.en.interfax.com.ua
www.en.interfax.com.ua
www.en.interfax.com.ua
www.en.interfax.com.ua
www.en.interfax.com.ua
www.en.interfax.com.ua
www.sputniknews.com
www.zakon.rada.gov.ua
www.zakon1.rada.gov.ua
www.pravda.com.ua
www.en.interfax.com.ua
www.unian.info
www.bbc.com
www.nytimes.com
www.bbc.com
www.bbc.com
www.theguardian.com
www.theguardian.com
www.usatoday.com
www.kyivpost.com
www.kyivpost.com
www.vybory.pravda.com.ua
www.ukrinform.net
www.reuters.com
www.usnews.com

Reference lists: 1
www.bbc.co.uk
www.theguardian.com
www.diariolarepublica.net
www.diariolarepublica.net
www.ojs.c3sl.ufpr.br
www.web.archive.org
www.espectador.com
www.latinamericanhistory.about.com
www.elhistoriador.com.ar
www.vice.com
www.web.archive.org
www.www0.parlamento.gub.uy
www.countrystudies.us
www.books.google.com
www.seattletimes.com
www.theguardian.com
www.edition.cnn.com
www.as-coa.org
www.google.com
www.es.noticias.yahoo.com
www.bbc.com
www.en.mercopress.com
www.web.archive.org
www.lemonde.fr
www.en.mercopress.com
www.lanacion.com.ar
www.web.archive.org
www.mercopress.info
www.elpais.com
www.es.noticias.yahoo.com
www.bbc.co.uk
www.world.time.com
www.nwww.koreaherald.com
www.peru21.pe
www.bbc.com
www.web.archive.org
www.elobservador.com.uy
www.gadebate.un.org
www.gadebate.un.org
www.un.org
www.medelu.org
www.bbc.com
www.bbc.com
www.nytimes.com
www.theguardian.com
www.economist.com
www.lr21.com.uy
www.web.archive.org
www.monocle.com
www.news.yahoo.com
www.eldiario.com.uy


Reference lists: 1
www.catholicnewsagency.com
www.catholicnews.com
www.freep.com
www.time.com
www.time.com
www.aodonline.org
www.aodonline.org
www.news.va
www.web.archive.org
www.press.vatican.va
www.web.archive.org
www.catholicnews.com
www.web.archive.org
www.themichigancatholic.com
317 Processing .... https://en.wikipedia.org/wiki/Diosdado_Cabello
Reference lists: 1
www.vicepresidencia.gob.ve
www.web.archive.org
www.informe21.com
www.avn.info.ve
www.eluniversal.com
www.elpais.com
www.newvision.co.ug
www.counterpunch.org
www.uk.reuters.com
www.ibtimes.com
www.controlciudadano.org
www.eltiempo.com
www.controlciudadano.org
www.wsj.com
www.web.archive.org
www.blythe.org
www.doctorpolitico.com
www.bbc.co.uk
www.news.bbc.co.uk
www.elpais.com
www.ipysvenezuela.org
www.cnpven.org
www.oas.org
www.sipiapa.org
www.bbc.co.uk
www.bbc.co.uk
www.diariouno.com.ar
www.conelmazodando.com.ve
www.wsj.com
www.oas.org
www.el-nacional.com
www.web.archive.org
www.amnestyusa.org
www.webcitation.org
www.asamb

Reference lists: 1
www.bos.zrc-sazu.si
www.bos.zrc-sazu.si
www.independent.co.uk
www.web.archive.org
www.mladina.si
www.nytimes.com
www.women.timesonline.co.uk
www.mladina.si
www.24ur.com
www.delo.si
www.web.archive.org
www.ukom.gov.si
www.mladina.si
www.24ur.com
www.dnevnik.si
www.independent.co.uk
www.web.archive.org
www.www2.gov.si
www.web.archive.org
www.rtvslo.si
www.slovenskenovice.si
www.tokio.veleposlanistvo.si
www.nytimes.com
www.legimonaco.mc
www.web.archive.org
www.slovak-republic.org
www.web.archive.org
296 Processing .... https://en.wikipedia.org/wiki/Youssef_Chahed
Reference lists: 1
www.kapitalis.com
www.aljazeera.com
www.aljazeera.com
www.businessnews.com.tn
www.leaders.com.tn
www.infodoc.agroparistech.fr
www.theses.fr
www.businessnews.com.tn
www.zeit.de
www.tunisia-live.net
www.majles.marsad.tn
www.aljazeera.com
www.nytimes.com
www.aljazeera.com
www.af.reuters.com
{'www.rtsh.al': 3, 'www.parlament.al': 2, 'www.telegrafi.com': 3, 'www.thecelticstar.com': 1, 'www.standar

,Source,Frequency
0,www.rtsh.al,3
1,www.parlament.al,2
2,www.telegrafi.com,3
3,www.thecelticstar.com,1
4,www.standard.al,1
...,...,...
9671,www.ukom.gov.si,1
9672,www.tokio.veleposlanistvo.si,1
9673,www.infodoc.agroparistech.fr,1
9674,www.theses.fr,1


In [501]:
sorted_freq_df = freq_df.sort_values(by='Frequency', ascending = False)
sorted_freq_df 

,Source,Frequency
7,www.web.archive.org,12850
147,www.nytimes.com,1397
56,www.theguardian.com,1362
42,www.reuters.com,1040
20,www.news.bbc.co.uk,916
...,...,...
4703,www.us.mobile.reuters.com,1
4701,www.tna.mcot.net,1
4697,www.fromtheold.com,1
4696,www.thaiembassy.sg,1


In [507]:
sorted_50_above = sorted_freq_df[sorted_freq_df['Frequency']>=100]

In [508]:
sorted_50_above

,Source,Frequency
7,www.web.archive.org,12850
147,www.nytimes.com,1397
56,www.theguardian.com,1362
42,www.reuters.com,1040
20,www.news.bbc.co.uk,916
86,www.bbc.com,837
120,www.bbc.co.uk,724
112,www.washingtonpost.com,628
11,www.books.google.com,563
54,www.telegraph.co.uk,454


In [512]:
sorted_50_above.shape[0]

55